# OOD Detection for Customer Intent Classification

## Resources

Datasets 
- [banking77](https://huggingface.co/datasets/banking77)
- [ATIS Airline Travel Information System](https://www.kaggle.com/datasets/hassanamin/atis-airlinetravelinformationsystem) 
- [Bitext - Customer Service Tagged Training Dataset for Intent Detection](https://github.com/bitext/customer-support-intent-detection-training-dataset)

Model
- [philschmid/BERT-Banking77](https://huggingface.co/philschmid/BERT-Banking77) 

BERT resources
- [BERT Neural Network - EXPLAINED!](https://www.youtube.com/watch?v=xI0HHN5XKDo) 
- [Transformer Neural Networks - EXPLAINED! (Attention is all you need)](https://www.youtube.com/watch?v=TQQlZhbC5ps)

OOD detection
- [`Todd`](https://github.com/icannos/Todd)
- [OOD Classification `ToddBenchmark`](https://github.com/icannos/ToddBenchmark/tree/master/examples/classifications) 

## Setup

### Import

In [4]:
import torch
from tqdm import tqdm

from typing import List

In [3]:
from Todd import ScorerType, MahalanobisScorer

In [1]:
from toddbenchmark.classification_datasets import prep_model
from toddbenchmark.classification_datasets_configs import (
    DATASETS_CONFIGS,
    load_requested_dataset,
)

from toddbenchmark.utils_classification import (
    prepare_detectors,
    evaluate_dataloader,
)

from toddbenchmark.utils import mk_file_name

### Load model and tokenizer

In [ ]:
MODELS = {"BERT": "philschmid/BERT-Banking77", "DistilBERT": "philschmid/DistilBERT-Banking77"} 

In [6]:
model, tokenizer = prep_model(MODELS["BERT"])

Downloading: 100%|██████████| 313/313 [00:00<00:00, 38.9kB/s]
c:\Users\pemma\AppData\Local\Programs\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pemma\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 232k/232k [00:00<00:00, 738k

In [ ]:
model 

### Load reference dataset

In [ ]:
train_loader, test_loader = load_requested_dataset(
    config_name="b77", 
    tokenizer=tokenizer, 
    train_size=100, 
    validation_size=0, 
    test_size=100
)

## Detectors

### Instantiate and fit

In [ ]:
detectors: List[ScorerType] = [MahalanobisScorer(layers=[-1])]

In [ ]:
detectors = prepare_detectors(detectors, model, train_loader, tokenizer)

### Evaluate

#### IN dataset

In [ ]:
print("Evaluating on the in-distribution training set")

records = evaluate_dataloader(
    model,
    train_loader,
    tokenizer,
    detectors
)

In [ ]:
print("Evaluating on the in-distribution test set")

records = evaluate_dataloader(
    model,
    test_loader,
    tokenizer,
    detectors
)